Keras models can be functional or sequential apis

Sequential model:
* simple,easy to use way to define models that consist of linear stacks of layers, layers are added to model in sequential order
*   output of each layer is fed as input to the next

Functional Model:
*  used to define more complex deep learning models with multiple inputs and outputs, or ones that share layers
*   you can also have branches, where instead of being fed to the next layer only, you can branch off
* non linear flows
* layers are defined as seperate objects, inputs,outputs are explicilty defined.


Use the mnist dataset to explore fucntional and sequential models



In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

(xtrain,ytrain),(xtest,ytest)=mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [2]:
print(xtrain.shape)
print(ytrain.shape)

# we are going to send the images to a NN, so lets flatten the values and also normalize between 0 and 1
xtrain=xtrain.reshape(-1,28*28).astype("float32")/255.0
xtest=xtest.reshape(-1,28*28).astype("float32")/255.0


print(xtrain.shape)
print(xtest.shape)

# you dont need to convert these numpy arrays, internally they will be converted to tensors

(60000, 28, 28)
(60000,)
(60000, 784)
(10000, 784)


In [3]:
from tensorflow import keras
from tensorflow.keras import layers
# Create a Sequential API -> very conveinent but not flexible (one input can map to one output)
# layers.dense is basically just a layer that has the inputs from previous layer, ie every node here is connected to every input from previous layer
# you can have as many of these as you would like

#in the model part you are basically defining the layers
model = keras.Sequential(
    [
        layers.Dense(512,activation='relu'),
        layers.Dense(256,activation='relu'),
        #below is the output layer, you can say here the softmax function
        layers.Dense(10)

    ]

)

# in the mode compile you are stating the optimizer and loss function and stuff

model.compile(
    #below if you use from_logits=True, you have to put this here, if you didnt put the
    #softmax in the output above, this will first send through a softmax and then to the sparsecategorical......
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

#specify the trianing stuff
model.fit(xtrain,ytrain,batch_size=32,epochs=5,verbose=2)

model.evaluate(xtest,ytest,batch_size=32,verbose=2)


Epoch 1/5
1875/1875 - 9s - loss: 0.1835 - accuracy: 0.9439 - 9s/epoch - 5ms/step
Epoch 2/5
1875/1875 - 4s - loss: 0.0796 - accuracy: 0.9747 - 4s/epoch - 2ms/step
Epoch 3/5
1875/1875 - 4s - loss: 0.0538 - accuracy: 0.9825 - 4s/epoch - 2ms/step
Epoch 4/5
1875/1875 - 4s - loss: 0.0410 - accuracy: 0.9869 - 4s/epoch - 2ms/step
Epoch 5/5
1875/1875 - 4s - loss: 0.0341 - accuracy: 0.9891 - 4s/epoch - 2ms/step
313/313 - 1s - loss: 0.0950 - accuracy: 0.9748 - 640ms/epoch - 2ms/step


[0.09504590183496475, 0.9747999906539917]

Batch Size:

-> basically batch size is the number of samples to be used for each iteration, so like if it is 32, you want 32 samples to train before the paramters are updated and you go to iteration 2

BatchMode: batch size is equal to total dataset, so the iterations and epochs are equivalent

mini batch mode: batch size is greater than 1 but less than the total dataset size

stochastic mode: the batch size is equal to 1, so paramters updated after every sample


-> iteration: an iteration is just when the weights are updated

-> epochs: basically when you train the dataset once, your gradient descent takes very small step, so one iteration of the current training data might not be enough, so you train the data through multiple epochs



In [4]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential(
    [
        keras.Input(shape=(28*28)),
        layers.Dense(512,activation='relu'),
        layers.Dense(256,activation='relu'),
        #below is the output layer, you can say here the softmax function
        layers.Dense(10)

    ]

)

print(model.summary())
#if you didnt add that input lin in the model above, you would not be able to see the model summary printed until after you compile


# in the mode compile you are stating the optimizer and loss function and stuff
model.compile(
    #below if you use from_logits=True, you have to put this here, if you didnt put the
    #softmax in the output above, this will first send through a softmax and then to the sparsecategorical......
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

#specify the trianing stuff
model.fit(xtrain,ytrain,batch_size=32,epochs=5,verbose=2)

model.evaluate(xtest,ytest,batch_size=32,verbose=2)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               401920    
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535818 (2.04 MB)
Trainable params: 535818 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


None
Epoch 1/5
1875/1875 - 5s - loss: 0.1868 - accuracy: 0.9427 - 5s/epoch - 3ms/step
Epoch 2/5
1875/1875 - 4s - loss: 0.0799 - accuracy: 0.9749 - 4s/epoch - 2ms/step
Epoch 3/5
1875/1875 - 4s - loss: 0.0543 - accuracy: 0.9830 - 4s/epoch - 2ms/step
Epoch 4/5
1875/1875 - 4s - loss: 0.0414 - accuracy: 0.9866 - 4s/epoch - 2ms/step
Epoch 5/5
1875/1875 - 4s - loss: 0.0335 - accuracy: 0.9894 - 4s/epoch - 2ms/step
313/313 - 1s - loss: 0.0879 - accuracy: 0.9774 - 598ms/epoch - 2ms/step


[0.08786247670650482, 0.977400004863739]

In [6]:
#Functional Apis

from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28*28))
x=layers.Dense(512,activation='relu', name='first_layer')(inputs)
x=layers.Dense(256,activation='relu')(x)
outputs=layers.Dense(10,activation='softmax')(x)
model=keras.Model(inputs=inputs,outputs=outputs)

model.compile(
    #below if you use from_logits=True, you have to put this here, if you didnt put the
    #softmax in the output above, this will first send through a softmax and then to the sparsecategorical......

    #becasue we made the softmax above, you can just set the logits=false or just remove it
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

#specify the trianing stuff
model.fit(xtrain,ytrain,batch_size=32,epochs=5,verbose=2)

model.evaluate(xtest,ytest,batch_size=32,verbose=2)


Epoch 1/5
1875/1875 - 5s - loss: 0.1872 - accuracy: 0.9428 - 5s/epoch - 3ms/step
Epoch 2/5
1875/1875 - 4s - loss: 0.0804 - accuracy: 0.9749 - 4s/epoch - 2ms/step
Epoch 3/5
1875/1875 - 4s - loss: 0.0557 - accuracy: 0.9821 - 4s/epoch - 2ms/step
Epoch 4/5
1875/1875 - 4s - loss: 0.0428 - accuracy: 0.9863 - 4s/epoch - 2ms/step
Epoch 5/5
1875/1875 - 4s - loss: 0.0323 - accuracy: 0.9895 - 4s/epoch - 2ms/step
313/313 - 1s - loss: 0.1236 - accuracy: 0.9675 - 622ms/epoch - 2ms/step


[0.1235736608505249, 0.9674999713897705]

In [8]:
# how to get outputs from certain layers -- works for both the sequential and functional api
#model.layers[-1] means last layer, [-2] means second last, etc

#the below overwrites the aove model
model=keras.Model(inputs=model.inputs,outputs=[model.layers[-2].output])

# you could also replace the [-2] with model.get_layer('layer_name).output

feature=model.predict(xtrain)
print(feature.shape)

1875/1875 [==============================] - 2s 1ms/step
(60000, 512)


In [12]:
# you could also get the output from every layer, will be in different matricies
from tensorflow import keras
from tensorflow.keras import layers

model=keras.Model(inputs=model.inputs,outputs=[layer.output for layer in model.layers])

features=model.predict(xtrain)

for feature in features:
  print(features.shape)

1875/1875 [==============================] - 3s 1ms/step


AttributeError: ignored